In [1]:
!pip install --upgrade pip
!pip install -q -U google-generativeai

In [2]:
import pandas as pd
import getpass
import json
import os
import csv
import time
import requests

# import google.generativeai as genai

In [3]:
gemini_api_key = getpass.getpass("Gemini API key: ")

Gemini API key: ········


In [4]:
%cd /content/drive/MyDrive/Colab/instate/

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab/instate/'
/Users/soodoku/Documents/GitHub/local_elections_up/scripts


In [5]:
df = pd.read_csv('../data/up_gp_sarpanch_2010_transliterate_prep.csv')

In [6]:
df.head()

,order,Attribute,Value
0,55362,district_name,संत रविदास नगर
1,55370,district_name,मिर्जापुर
2,11606,district_name,अलीगढ़
3,5297,district_name,ज्योतिबा फूले नगर
4,49659,district_name,मऊ


In [7]:
unique_names = df['Value'].to_list()

In [8]:
safety_settings = [
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  }
]


In [9]:
import requests
import json

API_KEY = gemini_api_key  # Replace with your actual API key
model_name = "gemini-pro"
#url = f"https://generativelanguage.googleapis.com/v1beta/models/{model_name}:generateContent"
url = f"https://generativelanguage.googleapis.com/v1/models/{model_name}:generateContent"

headers = {
    "Content-Type": "application/json",
    "x-goog-api-key": API_KEY
}


prompt = """
Transliterate the given text to English. Do not translate.
For instance, रानी is not queen but rani.
Respond in json format - {"key": "value"} where value is the transliterated text of key.
Example:
input:  'चंचला', 'रानी', 'दास', 'विकास'
output:  {'चंचला': 'Chanchala', 'रानी': 'Rani', 'दास': 'Das','विकास': 'Vikas'}

"""

In [10]:
data = {
    "contents": [{
        "parts":[
            {"text": prompt}
        ]
    }]
}

response = requests.post(url, headers=headers, data=json.dumps(data))

if response.status_code == 200:
    print(response.text)
else:
    print("Request failed:", response.status_code, response.text)

{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "{\n  \"चंचला\": \"Chanchala\",\n  \"रानी\": \"Rani\",\n  \"दास\": \"Das\",\n  \"विकास\": \"Vikas\"\n}"
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",
      "index": 0,
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE"
        }
      ]
    }
  ],
  "usageMetadata": {
    "promptTokenCount": 111,
    "candidatesTokenCount": 44,
    "totalTokenCount": 155
  }
}



In [11]:
def generate(prompt, debug=False):
    data = {
      "contents": [{
          "parts":[
              {"text": prompt}
          ]
      }]
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        if debug:
          print(response.text)
        res = json.loads(response.text)
        return res['candidates'][0]['content']['parts'][0]['text']
    else:
        return None

In [12]:
res = json.loads(response.text)

In [13]:
#res['candidates'][0]['content']['parts'][0]['text']

In [ ]:
translated_file = "../data/up_gp_sarpanch_2010_transliterate_out.csv"

# Function to divide names into chunks
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


# Function to get already processed names
def get_processed_names(file):
    if os.path.exists(file):
        with open(file, 'r') as f:
            reader = csv.DictReader(f)
            return {row['Name'] for row in reader}
    else:
        return set()

# Get already processed names
processed_names = get_processed_names(translated_file)
print(f"till now processed: {len(processed_names)}")

prompt = """
Transliterate the given text to English. Do not translate.
For instance, रानी is not queen but rani.
Respond in json format - {"key": "value"} where value is the transliterated text of key.
Example:
input:  'चंचला', 'रानी', 'दास', 'विकास'
output:  {'चंचला': 'Chanchala', 'रानी': 'Rani', 'दास': 'Das','विकास': 'Vikas'}

"""

with open(translated_file, 'a', newline='') as csvfile:
    fieldnames = ['Name', 'Transliterated']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # If file is empty write the header
    if csvfile.tell() == 0:
        writer.writeheader()

    c = 0
    # Iterate over names in chunks
    main_chunk = []
    for chunk in chunks(list(unique_names), 50):
        if c!=0 and c%100 == 0:
            print(f"done processing {c * 50}")
        c = c + 1

        # Skip names that have been processed
        chunk = [name for name in chunk if name not in processed_names]
        if not chunk:
            continue

        # Append the chunk to the main chunk
        main_chunk.extend(chunk)

        while True:
            try:
                # Make the API call
                # print(prompt + ', '.join(chunk))
                chat_response = generate(
                    prompt + ', '.join(main_chunk))
            except Exception as e:
                print(e)
                time.sleep(2)  # Sleep for a minute and then retry
                continue

            break  # If no errors, break the loop

        try:
          if chat_response:
            results = json.loads(chat_response)
            for name in results:
                writer.writerow({'Name': name, 'Transliterated': results[name]})
            csvfile.flush()  # Ensure data is written after each batch
        except Exception as e:
          print(e)
        time.sleep(1)  # Sleep for second
        main_chunk = []

till now processed: 5148
Unterminated string starting at: line 1 column 2 (char 1)
'content'
'content'
done processing 5000
'content'
'content'
done processing 10000
'content'
Expecting value: line 1 column 1 (char 0)
done processing 15000
Expecting value: line 1 column 1265 (char 1264)
Expecting ',' delimiter: line 1 column 1261 (char 1260)
done processing 20000
'content'
'content'
'content'
'content'


In [ ]:
datetime(2024, 5, 2)